In [1]:
#!pip install langchain cohere

In [2]:
# get a new token: https://dashboard.cohere.ai/

from getpass import getpass

COHERE_API_KEY = 'Hy75zdqpLP2Kon7HJEkMo1qLjQF6KKID8a8QM5mN'

In [3]:
# 1) tell the status of the order :info added
# 2) customer care call make a function that gives number that user can contact
# 3) refund policies :info added
# 4) status of refund :info added
# 5) damaged product :info added
# 6) delivery partner  :info added
# 7) available coupons :info added
# 8) terms and conditions :info added

In [4]:
# def give_customer_support_number():
#   customer_support_number = ""
#   return customer_support_number

# based on the user id add all orders and items the user ordered to the prompt
# information in the prompt will be sent to the chatbot
# variables that the chatbot can change are item refund status , damage status of item


In [5]:
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import datetime
import json
import re

###################
# input variables
target_user_id = 1
user_name = 'Alice'
#################

terms_conditions = open("terms_conditions.txt", "r").read()

refund_policies = open("refund_policies.txt", "r").read()

def return_coupon_information():
    # Load the JSON data from the "coupons.json" file
    with open("available_coupons.json", "r") as json_file:
        data = json.load(json_file)

    # Initialize an empty string to store the coupon information
    coupon_string = ""

    # Iterate through the categories and coupons
    for category in data["coupons"]:
        category_name = category["category"]
        coupon_string += f"Category: {category_name}\n"

        for coupon in category["coupons"]:
            coupon_name = coupon["name"]
            coupon_details = coupon["details"]
            coupon_string += f"Coupon Name: {coupon_name}\n"
            coupon_string += f"Coupon Details: {coupon_details}\n"
            coupon_string += "\n"
    return coupon_string

def update_user_info(user_id, text):
    # Define a regex pattern to extract the item name from the text
    item_name_pattern = r'<REFUND>(.*?)</REFUND>'
    
    # Find the item name using the regex pattern
    match = re.search(item_name_pattern, text)
    
    if match:
        item_name = match.group(1)
        
        # Load the customer_info.json file
        with open('customer_info.json', 'r') as file:
            data = json.load(file)
        
        # Update the refund_status to 'initiated' for the specified item
        for user in data['users']:
            if user['user_id'] == user_id:
                for item in user['items']:
                    if item['item_name'] == item_name:
                        item['refund_status'] = 'initiated'
                        item['order_status'] = 'waiting for return'
                        item['product_damaged'] = True
                        break
        
        # Save the updated data back to the customer_info.json file
        with open('customer_info.json', 'w') as file:
            json.dump(data, file, indent=4)
        
        print(f"Updated refund status for {item_name} in user {user_id}'s data.")
        return f"{item_name} refund and return process intiated."
    else:
        #print("Item name not found in the text.")
        return ""

def return_user_items(target_user_id):
    # Load the JSON data from the file
    with open('customer_info.json', 'r') as file:
        data = json.load(file)

    # Find the user with the specified user ID
    target_user = next((user for user in data['users'] if user['user_id'] == target_user_id), None)

    items_info = ""
    if target_user:
        user_name = target_user['user_name']
        for item in target_user['items']:
            item_name = item.get('item_name', 'Unknown Item')
            item_price = item.get('item_price', 'Unknown Price')
            order_status = item.get('order_status', 'Unknown Order Status')
            refund_status = item.get('refund_status', 'Unknown Refund Status')
            product_damaged = item.get('product_damaged', 'Unknown Product Damage Status')
            delivery_partner = item.get('delivery_partner_name', 'Unknown Delivery Partner')
            order_date = item.get('order_date', 'Unknown Order Date')
            delivery_date = item.get('delivery_date', 'Unknown Delivery Date')

            item_info = f"Item is {item_name} and its price is {item_price} dollars and its order status is {order_status} and its refund status is {refund_status} and its product damage status is {'damaged' if product_damaged else 'not damaged'} and Delivery Partner is {delivery_partner} and date on which order was placed is {order_date} {'and it was delivered on ' + delivery_date if delivery_date != 'N/A' else ''} ."

            items_info = items_info + item_info + "\n"
        return items_info
    else:
        print(f"User with user ID {target_user_id} not found.")

conversation_string = ''
while True:
    user_question = input()
    items_info = return_user_items(target_user_id=target_user_id)
    coupon_string = return_coupon_information()
    conversation_string = conversation_string + f"{user_name}: {user_question}\nMelissa:"

    # Get the current date
    current_date = datetime.date.today()
    # Format the date as "YYYY-MM-DD"
    formatted_date = current_date.strftime("%Y-%m-%d")

    template = """{terms_conditions}\n{refund_policies}\n{coupon_string}\n\n\nInstructions: You are a helpful customer support agent named Melissa, you love helping customers of the newly started and upcoming online store called Sparkles. You will give easy to understand, fun and cheerful responses to the user queries. You will be very positive, compassionate and helpful towards the user. If the user asks for a refund then tell the user you will initiate the process and at the end of the response generate '<REFUND> item name </REFUND>'  or both so that backend systems can use regex to find what needs to be updated in backend and make changes to database.\nInformation about the user\nUsername: {user_name}\nItems Ordered: \n{items_info}\nToday's Date: {formatted_date}\n{conversation_string}"""

    prompt = PromptTemplate(template=template, input_variables=['items_info', 'user_name', 'terms_conditions', 'refund_policies', 'coupon_string', 'formatted_date', 'conversation_string'])

    final_prompt = prompt.format(
        items_info=items_info, 
        user_name=user_name,
        terms_conditions=terms_conditions,
        refund_policies=refund_policies,
        coupon_string=coupon_string,
        formatted_date=formatted_date,
        conversation_string=conversation_string
    )

    print("Prompt being sent to the language model:")
    print(final_prompt)

    llm = Cohere(cohere_api_key=COHERE_API_KEY, stop=[f'{user_name}:'], temperature=1, model='command-nightly')
    llm_chain = LLMChain(prompt=prompt, llm=llm)

    response = llm_chain.run(items_info=items_info, user_name=user_name, terms_conditions=terms_conditions, refund_policies=refund_policies, coupon_string=coupon_string, formatted_date=formatted_date, conversation_string=conversation_string)
    print(response)
    conversation_string = conversation_string + response + '\n'
    refund_decision = update_user_info(user_id=target_user_id, text=response)
    print(refund_decision)
    if refund_decision != "":
        conversation_string = conversation_string + refund_decision + "\n"



Prompt being sent to the language model:
These terms and conditions for Sparkles encompass key elements of the online shopping process. Customers can place orders using various payment methods, with prices subject to change. Shipping options and delivery times are offered, and customers are responsible for accurate shipping information. Returns and refunds can be requested for eligible items within a specified period, with special conditions for certain product categories. Product information is provided for reference, but accuracy is not guaranteed, and customer data is handled following the Privacy Policy. Users are responsible for their account security, and intellectual property is protected. Disputes can be resolved through customer support, and changes to terms and conditions will be notified. For questions or assistance, customers can contact Sparkles' customer support. Please note that these terms and conditions are a general overview, and it's essential to consult legal expert